In [14]:
import pandas as pd
import numpy as np
import os
import glob
from tasks import compute_index
from tasks.post_process import compare_2020_2019_data_report

# For the results

In [15]:
def make_2019_2020_correlation_report():
    
    data_2019 = pd.read_csv('data/2019_archive/result.csv').assign(version='v_2019')
    data_2020 = pd.read_csv('data/full_data/result.csv').assign(version='v_2020')
    data = pd.concat([data_2019, data_2020], axis=0).dropna(subset=['Value'])
    ISO_with_index = data.query("Aggregation == 'Index'").dropna().ISO.unique() # Select only ISOs where the full index is computed to remove some noise
    data = data[data.ISO.isin(ISO_with_index)]
    
    pivoted_data = data.pivot(index=['Variable', 'Aggregation', 'ISO', 'Year'], columns=['version'], values='Value')
    
    corr_by_var = pivoted_data.groupby(['Variable', 'Aggregation']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')
    corr_by_var_ISO = pivoted_data.groupby(['Variable', 'Aggregation', 'ISO']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')


    return pivoted_data, corr_by_var.reset_index(), corr_by_var_ISO.reset_index()



In [16]:
data, corr_by_var , corr_by_var_ISO = make_2019_2020_correlation_report()

In [17]:
corr_by_var.query('Aggregation == "Indicator" and corr < 0.9')

,Variable,Aggregation,corr
47,GE1,Indicator,0.798867
51,GE3,Indicator,0.898253
58,GN1,Indicator,0.209781
76,SE2,Indicator,0.438966


In [18]:
corr_by_var.query('Aggregation == "Indicator_normed" and corr < 0.9')

,Variable,Aggregation,corr
48,GE1,Indicator_normed,0.770216
59,GN1,Indicator_normed,0.584670
83,SL1,Indicator_normed,0.695073


In [19]:
corr_by_var.query('Aggregation == "Category" and corr < 0.9')

,Variable,Aggregation,corr
57,GN,Category,0.584670
81,SL,Category,0.762107


In [20]:
data.loc['Index', :, :, 2020].sort_values(by="v_2020").tail(30)

version                          v_2019     v_2020
Variable Aggregation ISO Year                     
Index    Index       FRA 2020.0   69.08  66.538505
                     POL 2020.0   68.61  67.009597
                     HRV 2020.0   67.77  67.403858
                     NLD 2020.0   67.20  67.797355
                     SVN 2020.0   71.00  67.923544
                     PRT 2020.0   70.36  67.925303
                     ROU 2020.0   67.97  68.094001
                     CHE 2020.0   73.35  68.341022
                     LVA 2020.0   69.29  68.476742
                     HUN 2020.0   70.93  68.750482
                     ITA 2020.0   68.03  69.738354
                     LTU 2020.0   71.97  70.594453
                     EST 2020.0   69.96  72.200821
                     DEU 2020.0   75.81  72.837222
                     SVK 2020.0   74.19  73.108257
                     AUT 2020.0   75.43  73.405498
                     FIN 2020.0   74.89  74.418260
                     CZE 2020.0   76.96  74.715992
                     DNK 2020.0   76.54  75.924032
                     SWE 2020.0   78.54  78.322470
                     AFG 2020.0   27.58        NaN
                     BIH 2020.0   39.30        NaN
                     CPV 2020.0   47.15        NaN
                     DZA 2020.0   28.00        NaN
                     IRN 2020.0   48.47        NaN
                     MLT 2020.0   31.76        NaN
                     MNE 2020.0   43.79        NaN
                     TJK 2020.0   29.03        NaN
                     TTO 2020.0   30.30        NaN
                     ZMB 2020.0   31.73        NaN

In [21]:
data.loc[:, 'Category', 'AFG', 2020]

version,v_2019,v_2020
Variable,,
AB,43.83,36.555367
BE,9.21,8.575380
CV,37.81,36.818737
EE,70.57,68.108864
EQ,71.19,71.996948
EW,NaN,21.173672
GB,40.11,40.109690
GE,79.24,78.489210
GJ,1.00,1.000000


In [ ]:
test = pd.read_csv('data/full_data/data.csv')

In [ ]:
test.query("Indicator == 'ME1' and ISO == 'FRA'")

In [ ]:
pd.read_csv('data/indicator/SL1/processed/SL1_origin.M.csv').query("ISO == 'NLD'")

In [9]:
data.loc['GN1', :, 'FRA', ]

version                               v_2019     v_2020
Variable Aggregation      ISO Year                     
GN1      Indicator        FRA 2005.0    0.02   0.017971
                              2006.0    0.02   0.018913
                              2007.0    0.02   0.019283
                              2008.0    0.02   0.019776
                              2009.0    0.02   0.020812
                              2010.0    0.02   0.021638
                              2011.0    0.02   0.022371
                              2012.0    0.02   0.022317
                              2013.0    0.02   0.022403
                              2014.0    0.02   0.022437
                              2015.0    0.02   0.022429
                              2016.0    0.02   0.022020
                              2017.0    0.02   0.021233
                              2018.0    0.02   0.020047
                              2019.0    0.02   0.019435
                              2020.0    0.02   0.019435
         Indicator_normed FRA 2005.0   35.74  32.835616
                              2006.0   37.50  34.736817
                              2007.0   38.19  35.150461
                              2008.0   39.10  36.148657
                              2009.0   41.12  38.252856
                              2010.0   42.71  39.927779
                              2011.0   44.08  41.415188
                              2012.0   43.95  41.306405
                              2013.0   44.16  39.101452
                              2014.0   44.31  39.159812
                              2015.0   44.36  38.750553
                              2016.0   43.55  36.709265
                              2017.0   42.02  34.296180
                              2018.0   42.02  30.435878
                              2019.0   42.02  29.660832
                              2020.0   42.02  29.660832

In [ ]:
data.loc['ME1', :, 'USA', ]

In [ ]:
data.loc['SL1', :, 'USA', 2020]

In [ ]:
data.loc[:, 'Dimension', 'USA', 2020]

# SL1:

Problem from last year's version corrected today (Large outliers that where not accounted for and that are now removed.)


New data includes less points some of the values are not computed (eg the outliers, HKG, SGP etc)

**Conclusion**: Using SL1 v_2020 seems more appropriate 

In [ ]:
data.loc['SL1', 'Indicator_normed', : ,2019]['v_2019'].sort_values().head(15)

# GE1

Values are different from last year despite multiple APIs check. It is unclear why the values are different.

TO CHECK but not clear what's going on

In [ ]:
corr_by_var_ISO.query('Variable == "GE1" and Aggregation == "Indicator"').sort_values(by='corr').head(10)

In [ ]:
pd.read_csv('data/indicator/GE1/preprocessed/GE1.0_CW.csv')#.query('ISO == "FRA"')

In [ ]:
pd.read_csv('data/indicator/GE1/computed/GE1.csv').query('ISO == "FRA"')

In [ ]:
data.loc['GE1', 'Indicator', 'FRA']

In [ ]:
data.loc['GE1', 'Indicator', 'ITA' , :]#['v_2019'].sort_values().head(15)

In [ ]:
config = {
            "GGI_code": "GE1.0",
            "params": {
              "gas_ids[]": 326,
              "sector_ids[]": [1299],
              "source_ids[]": [111]
            }
        }

In [ ]:
from download.downloaders import download, CW_Downloader
from processing.api_preprocessors import CW_Preprocessor

In [ ]:
test = download(API_name="CW", path=None, config=config)
test = CW_Preprocessor('GE1.0').json_to_pandas(test['data'])

In [ ]:
pop = pd.read_csv('data/indicator/GE1/processed/GE1.1_WB.csv')

In [ ]:
(test.query('iso_code3== "FRA"').set_index('year').value * 1e6 / pop.query('ISO == "FRA"').set_index('Year')['Value']).dropna()

In [ ]:
data.loc['GE1', 'Indicator', 'FRA']

In [ ]:
pd.read_csv('data/indicator/GE1/processed/GE1.0_CW.csv')

In [ ]:
def process_GE1_0(df):
    df = df.copy()
    piv = df.pivot(index=['iso_code3', 'year'], columns=['sector'], values='value')
    piv['value'] = (piv['Total excluding LUCF'] - piv['Agriculture']) * 1e6
    piv = (
        piv['value'].reset_index().assign(Description='Total excluding LUCF and Agriculture')
                    .assign(data_source='CAIT _and_ PIK')
    )
    
    
    return piv.query('value > 0')

In [ ]:
process_GE1_0(test)

# GE3 

Same as GE1, not clear what the problem is and where it comes from


In [ ]:
config = {
            "GGI_code": "GE3.1",
            "params": {
                "gas_ids[]": [
                    327,
                    328,
                    329
                ],
                "sector_ids[]":[
                    1304,
                    1302
                ],
                "source_ids[]": 111
            }
        }


test = download(API_name="CW", path=None, config=config)


In [ ]:
test = CW_Preprocessor('GE2.0').json_to_pandas(test['data'])

In [ ]:
test

In [ ]:
CW_Downloader('https://www.climatewatchdata.org/api/v1/data/historical_emissions').get_CW_API_ids("sectors")

#  ME1 

Check for a different source

In [ ]:
data.loc['ME1', 'Indicator', 'RWA']

# GN1 

Values are one the right order of magnitude but change from one version to the other, should double check properly and redownload.

In [ ]:

def make_2019_2020_correlation_report_bis():
    
    data_2019 = pd.read_csv('data/2019_archive/data.csv').assign(version='v_2019')
    data_2020 = pd.read_csv('data/full_data/data.csv').assign(version='v_2020')
    data = pd.concat([data_2019, data_2020], axis=0).dropna(subset=['Value'])
    
    pivoted_data = data.pivot(index=['Indicator', 'ISO', 'Year'], columns=['version'], values='Value')
    
    corr_by_var = pivoted_data.groupby(['Indicator']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')
    corr_by_var_ISO = pivoted_data.groupby(['Indicator', 'ISO']).apply(lambda x: x[['v_2019', 'v_2020']].dropna().corr().values[0, 1]).to_frame(name='corr')


    return pivoted_data, corr_by_var.reset_index(), corr_by_var_ISO.reset_index()
data, corr_by_var , corr_by_var_ISO = make_2019_2020_correlation_report_bis()

In [ ]:
corr_by_var.query('corr < 0.9')

In [ ]:
corr_by_var_ISO.query("Indicator == 'GN1'").sort_values(by='corr').dropna().head(20)

In [ ]:
data.loc['GN1', 'ITA']

In [ ]:
corr_by_var_ISO.query('Variable == "GN1" and Aggregation == "Indicator"').sort_values(by='corr').dropna().tail(20)

# SE2

In [ ]:
corr_by_var_ISO.query('Variable == "SE2" and Aggregation == "Indicator"').sort_values(by='corr').head(30)

In [ ]:
data.loc['SE2', 'Indicator', 'RWA']